In [2]:
import pandas as pd
# from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import FirefoxOptions 
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# import time
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database

In [3]:
url = 'https://racetozerodataexplorer.org/proceed/'

scope1_lst = []
scope2_local_lst = []
scope2_market_lst = []
name_lst = []

master_lst = [scope1_lst, scope2_local_lst, scope2_market_lst, name_lst]

options = FirefoxOptions()
options.add_argument("--headless") # comment out to see how it works

with webdriver.Firefox(options=options) as driver:
    driver.get(url)
    headers = driver.find_elements(By.XPATH, "//*[@class= 'styles_table__jWPKo']/thead/tr/th")
    headers = [header.text for header in headers]
    table = driver.find_elements(By.XPATH, "//*[@class= 'styles_table__jWPKo']/tbody/tr")
    
    for row in table:
        name = row.find_element(By.TAG_NAME, 'th')
        # print(name.text)
        master_lst[3].append(name.text)
        for i, cell in enumerate(row.find_elements(By.TAG_NAME, 'td')):
            if i % 9 in [0,1,2]: # only interested in first 3 columns
                try:
                    cell.find_element(By.CLASS_NAME, 'styles_button__IS06I').click()
                    WebDriverWait(driver, 10).until( # Wait for pop-up to load
                        EC.presence_of_element_located((By.CLASS_NAME, 'styles_axisValue__ipOq1')))
                    values = driver.find_elements(By.CLASS_NAME, 'styles_axisValue__ipOq1')
                    master_lst[i].append([value.text.replace(',','') for value in values])
                    driver.find_element(By.CLASS_NAME, 'styles_close__cpVjT').click()
                except:
                    master_lst[i].append(['', '', '', ''])

37 Interactive Entertainment
A.P. Moller - Maersk
AAK AB
AB Volvo
ABB
AbbVie Inc
Accenture
Accor
adidas AG
ADIENT plc
Adobe
AECOM
AENA SME SA
AFRY AB
Agder Energi
Agilent Technologies Inc.
Ahold Delhaize
Ajinomoto Co.Inc.
Akamai Technologies Inc
Akzo Nobel NV
Alfa Laval Corporate AB
Alibaba Group Holding Limited
Amazon
Ambev S.A
Ambuja Cements
America Movil SAB de CV
American Airlines Group Inc
American Axle & MFG Holdings Inc
American Eagle Outfitters Inc.
Americanas S.A.
Amey UK plc
Analog Devices, Inc.
Apple Inc.
Applied Materials Inc.
Aramark Corporation
Arcadis
ARÇELİK A.Ş.
Ardagh Group SA
Asahi Group Holdings, Ltd.
Ashland LLC
Asics Corporation
Askul Corporation
ASOS
Assa Abloy
AstraZeneca
Atea ASA
Atlantia
Atlassian Corporation Plc
Atos SE
Aurubis AG
Autodesk, Inc.
Autoliv Inc
Autostrade per l'italia Spa
Axiata Group Berhad
Azul Sa
Babcock International Group PLC
BAE Systems
Balfour Beatty
Barratt Developments plc
Bayer AG
BCE Inc.
Becton, Dickinson and Co.
Beiersdorf AG
Bekaert

In [95]:
headers = [header.strip().replace('\n', ' ').replace(
    '4 years trend (ending in reporting year)', '') for header in headers]
years = ['2018', '2019', '2020', '2021']
    
# Need to merge all these 
df_scope1 = pd.DataFrame(master_lst[0], index=master_lst[3], columns=years).reset_index(drop=False)
df_scope2_loc = pd.DataFrame(master_lst[1], index=master_lst[3], columns=years).reset_index(drop=False)
df_scope2_mkt = pd.DataFrame(master_lst[2], index=master_lst[3], columns=years).reset_index(drop=False)

In [37]:
engine = create_engine("sqlite:///GDS.db")
if not database_exists(engine.url):
    create_database(engine.url)

# Add dataframes to database to avoid having to rerun webscraping each time
for df, header in [(df_scope1, headers[1]), (df_scope2_loc, headers[2]), (df_scope2_mkt, headers[3])]:
    df.rename(columns={'index': 'Company Name'}, inplace=True)
    df.to_sql(header.replace(' ', '_'), engine, if_exists='replace', index=False)

In [56]:
df_scope1 = pd.read_sql_table('Scope_1_emissions_(tCO₂e)', engine)
df_scope2_loc = pd.read_sql_table('Scope_2_emissions_(tCO₂e)_Location', engine)
df_scope2_mkt = pd.read_sql_table('Scope_2_emissions_(tCO₂e)_Market', engine)

In [57]:
# Drop Interpublic Group of Companies, Inc. which is duplicated in data source
df_scope1 = df_scope1.loc[~df_scope1.index.duplicated()] 
df_scope2_loc = df_scope2_loc.loc[~df_scope2_loc.index.duplicated()] 
df_scope2_mkt = df_scope2_mkt.loc[~df_scope2_mkt.index.duplicated()] 

In [58]:
df_scope1 = df_scope1.melt(id_vars='Company Name', value_name='Scope_1_emissions_(tCO₂e)', var_name='Year')
df_scope2_loc = df_scope2_loc.melt(id_vars='Company Name', value_name='Scope_2_emissions_(tCO₂e)_Location', var_name='Year')
df_scope2_mkt = df_scope2_mkt.melt(id_vars='Company Name', value_name='Scope_2_emissions_(tCO₂e)_Market', var_name='Year')

In [60]:
df = df_scope1.merge(df_scope2_loc, on=['Company Name', 'Year']).merge(df_scope2_mkt, on=['Company Name', 'Year'])

df = df.loc[~df.duplicated()] # Remove companies with no data whatsoever

In [99]:
companies_sample = pd.DataFrame(df['Company Name'].unique(), columns=['Company Name']).sample(100, random_state=1234)

df = df.merge(companies_sample, on='Company Name', how='inner')
df.to_sql('Emissions_Data_Joined', engine, if_exists='replace', index=False)

400

In [103]:
df.loc[df['Year']=='2018'] # Allows us to access data for each time period like this

,Company Name,Year,Scope_1_emissions_(tCO₂e),Scope_2_emissions_(tCO₂e)_Location,Scope_2_emissions_(tCO₂e)_Market
0,37 Interactive Entertainment,2018,-,-,
4,Accor,2018,280706,1829278,-
8,AECOM,2018,-,-,
12,AFRY AB,2018,1708,2347,1991
16,Amazon,2018,-,-,
...,...,...,...,...,...
380,Vodafone Group,2018,257887,1908842,1737251
384,Western Power Distribution plc,2018,,,
388,Winnebago Industries Inc,2018,,,
392,WPP Group,2018,7309,95144,73830
